# Membership Inference Attacks against ML models

Resource:  [Memebership Inference Attacks against ML models - Reza Shokri](https://arxiv.org/pdf/1610.05820.pdf)

## Basic Libraries Imports

We will use the CIFAR-10 dataset

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.io import read_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

!pip install mia
import mia

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data

In [ ]:
# configurations 
BATCHES = 200
EPOCHS = 10
LEARNING_RATE = 0.001 

## Train a target model

In [ ]:
# Load data (CIFAR-10)


def load_dataset(batch_size) :
  """Get CIFAR-10 data, apply regularization and one-hot-encoding and return them"""  
  # get a transformer
  transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  # get the training set
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
  # get the test set
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

  # create the dataloaders
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

  return trainloader, testloader  

In [ ]:
trainloader, testloader = load_dataset(BATCHES)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# set up simple CNN
class Net(nn.Module):
  def __init__(self, trainloader, epochs):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    self.epochs = epochs

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1) # flatten all dimensions except batch
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def train(self,_optimizer):
    criterion = nn.CrossEntropyLoss()
    optimizer = _optimizer
    
    for epoch in range(self.epochs):  # loop over the dataset multiple times

      running_loss = 0.0
      for i, data in enumerate(trainloader, 0):
      # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

    print('Finished Training')

In [ ]:
# create the model
net = Net(trainloader, EPOCHS)

In [ ]:
# train the model
net.train(optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=0.9))

Finished Training


In [ ]:
outputs = net(images)

NameError: ignored

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))